### You will meet with your instructors on Monday to “pitch” your project idea. Be prepared to answer the following questions:

What questions will you be addressing?

What ways might you visualize this data to address the question?

You will meet with the instructors to discuss your progress on the project and your plan for the rest of the week to make sure you are on the right track. We will be focusing on the steps of the CRISP-DM model. Be prepared to answer the following questions.

#### Business Understanding
* Have you clearly defined your goal for your analysis?
    * <font color='green'>Gain insight on housing affordability.</font>
    * <font color='green'>Determine which factors have the largest effect on housing affordability.</font>
    * <font color='green'>What are the number of counties that fit within the range of housing affordability?</font>

* Have you thought about who your audience is and how they would use this information?
    * <font color='green'>Washington State Housing Finance Commission, they will use our model to help them improve affordability.</font>

* How does this help the goals of the business/organization?
    * <font color='green'>This will give them insight to inform their policy and tax decisions as well as to create programs to help people afford homes.</font>

#### Data Understanding
* What data are you using?
    * the king county house data provided
    * equation for housing affordability from an industry standard
* How does your data help you answer the business question?
    * Provide a linear regression model...
    * Tell them which counties are more affordable than others.
    * Tell them which counties have a more serious affordabiliy problem.
* How many observations does your dataset contain?
    * 21,597 not including rows with NAN for waterfront, view, or yr_renovated
* What is the distribution of your data?
    * see plots, mostly 
* What data types do you have?
    * ints, floats, dates/years

#### Data Preparation
* Have you looked/dealt with missing values?
    * waterfront, view, yr_renovated (possibly just never renovated?)
* Have you done any data-type conversion?
    * ex: numerical data incorrectly ‘typed’ as strings.
    * will need to convert sqft_basement from object to int or float
* Does your data contain any outliers or non-sensical values?
    * There are outliers but we need to revisit
    
    
<font color='green'>Given income and desired features of the house, 
affordable zip code --> 
website with sliders

fit a linear regression model
increase variable 1 unit, sales goes up x $'s
Provide a function takes in zip code, income, and important features (allow assignment to 10-ish variables, assign average for other variables), return a price prediction, mortgage, min, max and 5 closest houses sold in 2014-2015.